# Projekt Algorytmy Uczenia Maszynowego

## Cel Projektu
Celem tego projektu jest wykorzystanie algorytmów uczenia maszynowego do zrealizowania zadania klasyfikacji.
## Przedstawienie problemu
W ramach projektu zostanie wykonane zadanie klasyfikacji zdjęć zwierząt. Przygotowana implementacja powinna rozpoznawać 
zwierzęta z pięciu różnych klas, w tym przypadku będą to:
* kurczak
* owca
* koń 
* pająk
* słoń

Do realizacji zadania wykorzystano zbiór danych dostępny pod linkiem:
https://www.kaggle.com/datasets/alessiocorrado99/animals10
## Wybrane algorytmy uczenia maszynowego
Jednym z założeń projektowych jest zrealizowanie danego zadnia z wykorzystaniem trzech podstawowych 
algorytmów uczenia maszynowego, w tym przypadku zdecydowano się na wybranie następujących algorytmów:
* Maszyna wektorów nośnych
* Wielowarstwowy perceptron 
* K najbliższych sąsiadów 

Po wykorzystaniu algorytmów uczenia maszynowego należało zaimplementować mechanizm łączenia wykorzystywanych algorytmów.
W tym projekcie zdecydowano się na wykorzystanie `głosowania większościowego`.   

## Wczytanie oraz wizualizacja zbioru danych  
Jednym z kluczowych elementów w zadania klasyfikacji jest dobranie odpowiedniego zbioru danych.
W tym przypadku zaszła potrzeba usunięcia części elementów ze zbioru ze względu na ich jakość.
Zdjęcia w których występowało wiele zwierząt, były bardzo zaszumione albo przedstawiały nie te 
zwierzęta musiały zostać usunięte. W efekcie końcowym zbiory danych zostały zmniejszone do 250 
zdjęć dla każdej klasy. Mimo zmniejszenia zbioru uczącego uzyskano lepsze rezultaty.  


In [ ]:
from skimage.io import imshow, imread_collection
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np


def load_images(class_name, values, labels):
    path_to_image = 'raw-img/'
    postfix = '/*.jpeg'
    image_size = (128,128)
    for image in imread_collection(path_to_image + class_name + postfix):
        values.append(resize(image, image_size))
        labels.append(class_name)

def create_data_set():
    class_names = ['spider', 'horse', 'elephant', 'chicken', 'sheep']
    values = []
    labels = []
    for class_name in class_names:
        load_images(class_name, values, labels)

    return np.array(values), np.array(labels)

def plot_data_set(x, y, unique_labels):
    fig, axes = plt.subplots(1, len(unique_labels))
    fig.set_size_inches(15,4)
    fig.tight_layout()

    for ax, label in zip(axes, unique_labels):
        ax.imshow(x[np.where(y == label)[0][0]])
        ax.axis('off')
        ax.set_title(label)

x, y = create_data_set()
unique_labels = np.unique(y)

plot_data_set(x, y, unique_labels)


## Ekstrakcja cech

Niezbędnym elementem w wykorzystaniu algorytmów uczenia maszynowego do zadania klasyfikacji jest wyciągniecie 
najważniejszych informacji ze zbioru uczącego. W przypadku danych jakimi są obrazy można zrobić to na wiele sposobów, 
miedzy innymi:
* wykorzystać histogram
* wykorzystać operatory pozwalające uwypuklić krawędzie
* wykorzystać bardziej skomplikowane algorytmy takie jakie np. transformacja HOGa czy cechy Haar

Do realizacji tego zadania zdecydowano się na wybranie transformaty `HOG`.

### Transformacja HOG(Histogram zorientowanych gradientów)

Opis cech wykorzystujący histogram zorientowanych gradientów, wymaga wykonania wielu pośrednich operacji 
aby uzyskiwać oczekiwane rezultaty. 

##### Wyznaczenie gradientów

Pierwszym krokiem jest wyznaczanie gradientów z w pionie oraz w poziomie wykorzystując filtrując z wykorzystaniem jąder:

$$ [-1, 0, 1] \text{ oraz }  [-1, 0, 1]^T$$

##### Kierunek oraz natężenie gradientów

Kolejnym krokiem jest obliczenie natężenie gradientów oraz ich kierunków zgodnie z wzorami


$$
    g = \sqrt{g^2_x + g^2_y}
$$

$$
    \theta = arctan\frac{g_y}{g_x}
$$

#### Obliczanie histogramów gradientów

Kolejnym elementem było obliczanie gradientów, aby to zrobić obraz należy podzielić na komórki a następnie dla każdej 
z komórek dodać wartości gradientów aby uzyskać histogram dla danej ilości kierunków.

#### Normalizacja gradientów

Ostatnim elementem jest normalizacja gradientów. Normalizacja gradientów polega na dobraniu bloków złożonych z komórek
a następnie wykonania dla niej normalizacji w taki sposób jak dla wektorów.


### Wykorzystanie transformacji HOG

W ramach projektu wykorzystano gotową implementację pozwalającą z biblioteki scikt-image na dostarczenie deskryptorów HOG.
Przy czym najważniejsze parametry jakie zostały wykorzystane to:

* orientation — ilość kierunków gradientów branych pod uwagę
* pixels_per_cell — ilość pikseli w jednej komórce 
* cells_per_block — ilość komórek w jednym bloku

Wyżej wymienione parametry zostały dobrane z wykorzystaniem przeszukania (`GridSearchCV`), tak aby dostarczyć
klasyfikatorom najlepszy zestaw cech.

Aby przedstawić efekt działania transformaty HOG można wykorzystać parameter `visualize`, dzięki któremu można zwrócić
wizualny efekt działania operacji co zostało przedstawione poniżej.


Ostatnim elementem poniższego bloku jest podzielenie zbioru(z wykorzystaniem funkcji `train_test_split`) na część testową
oraz część treningową w proporcji jeden do cztery. Przy podziale wykorzystano parameter `shuffle`, który pozwala na
wstępne przetasowanie danych. Wykorzystano także parametr `random_state`, który pozwala na podział danych 
w powtarzalny sposób.  




In [ ]:
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

def extract_hog_features(values, orientation = 11, pixels_per_cell=(8,8), cells_per_block=(7,7)):
    scalify = StandardScaler()
    return scalify.fit_transform(
        np.array([ hog(img, orientations=orientation, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, channel_axis=-1) for img in values ]))

def plot_hog_features_extraction(x, y, unique_labels, orientation = 9, pixels_per_cell=(9,9), cells_per_block=(4,4)):
    fig, axes = plt.subplots(1, len(unique_labels))
    fig.set_size_inches(15,4)
    fig.tight_layout()

    for ax, label in zip(axes, unique_labels):
        fd, img = hog(
            image=x[np.where(y == label)[0][0]],
            orientations=orientation,
            pixels_per_cell=pixels_per_cell,
            cells_per_block=cells_per_block,
            visualize=True,
            channel_axis=-1) 
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(label)

hog_train = extract_hog_features(x)
plot_hog_features_extraction(x, y, unique_labels)

x_train, x_test, y_train, y_test = train_test_split(hog_train, y, test_size=0.2, shuffle=True, random_state=15)


## Wykorzystanie maszyny wektorów nośnych(SVM) w zadaniu klasyfikacji

Algorytm `SVM` można potraktować jako rozwinięcie modelu perceptronu. Wykorzystanie algorytmu perceptronu daje możliwość minimalizowania błędu klasyfikacji, z kolei podstawowym celem SVM jest maksymalizacja marginesu. Przy czym margines jest odległością pomiędzy hiperprzestrzenią rozdzielającą(granicą decyzyjną) a najbliższymi próbkami uczącymi(`wektorami nośnymi`). Wykorzystując algorytm SVM dążymy do uzyskania szerokich granic decyzyjnych, ponieważ takie modele są bardziej odporne na błędy uogólnienia, natomiast możliwie jest wystąpienie przetrenowanie. 

W przypadku algorytmu ważnym parametrem jest  `C`, sterując wartością `C` można kontrolować karę za niewłaściwą klasyfikację. Duże wartości `C` zwiększają kary za błędną klasyfikację, natomiast małe wartości `C` zmniejszają kary. Im większy jest parametr `C` tym bardziej generalizujmy model jednocześnie zmniejszamy margines, małe wartości `C` mogą doprowadzić do przetrenowania modelu. Wartość parametru `C` została dobrana za wykorzystaniem przeszukania GridSearchCV.

Kolejnym istotnym elementem algorytmu `SVM` jest dobranie odpowiedniego jądra. Zadanie klasyfikacji dla danych rozdzielnych z wykorzystaniem liniowej hiperpłaszczyzny jest trywialnym zadaniem z punktu widzenia algorytmu `SVM`. W momencie w którym dane nie są rozdzielne w sposób liniowy sytuacja jest gorsza, jednak zastosowanie odpowiedniej funkcji jądrowej pozwala na utworzenie nieliniowych kombinacji pierwotnych cech, które finalnie mogą być mapowane na przestrzenie o większej liczbie wymiarów, w których to będą liniowo separowalne. Po przeanalizowaniu dostępnych przypadków wybrano funkcje `RBF`(radial basis function).



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import svm

svm_svc = svm.SVC(C=1.1, random_state=1)
svm_svc.fit(x_train, y_train)

train_labels_predicted = svm_svc.predict(x_train)
test_labels_predicted = svm_svc.predict(x_test)


print('SVM percentage correct for test data: ', 100*accuracy_score(train_labels_predicted, y_train))
print(classification_report(y_train, train_labels_predicted))
print(confusion_matrix(y_train, train_labels_predicted))


print('SVM percentage correct for test data: ', 100*accuracy_score(test_labels_predicted, y_test))
print(classification_report(y_test, test_labels_predicted))
print(confusion_matrix(y_test, test_labels_predicted))


## Wyniki Algorytmu SVM

Jak można zaobserwować algorytm dla zbioru uczącego ma dokładność na poziomie 100 procent. Dla zbioru testowego wydajność jest już gorsza natomiast jest powyżej 84 procent co do daje względnie dużą dokładność. Najgorsze wyniki klasyfikacji zostały otrzymane dla konia natomiast najlepsze dla kurczaka oraz pająka.  

## Krzywa uczenia algorytmów ucznia maszynowego

Jednym z elementów pomocniczych w trakcie wykorzystywania algorytmów ucznia maszynowego jest krzywa uczenia. 
Przez analizę krzywej uczenia można obserwować, czy model nie został przetrenowany, jak skaluje się model
oraz jakie wyniki uzyskuje. Poniżej zostało zaimplementowane wyświetlanie krzywych ucznie z wykorzystaniem 
walidacji krzyżowej.


In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

def plot_learning_curve(
    estimator,
    X,
    y,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    
    _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title('Learning curves')
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)
   
    # Plot learning curvea
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

cv = ShuffleSplit(n_splits=3, test_size=0.2, random_state=15)


## Poniżej zostały przedstawione krzywe uczenia dla algorytmu SVM

In [ ]:
plot_learning_curve(svm_svc, hog_train, y, cv=cv, n_jobs=4)

# Wykorzystanie perceptronu wielowarstwowego(MLP) w zadaniu klasyfikacji

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_classifier = MLPClassifier(alpha=1, max_iter=1000)
mlp_classifier.fit(x_train, y_train)

labels_predicted_mlp = mlp_classifier.predict(x_test)

print('MLP percentage correct: ', 100*accuracy_score(labels_predicted_mlp, y_test))
print(classification_report(y_test, labels_predicted_mlp))
print(confusion_matrix(y_test, labels_predicted_mlp))

## Poniżej zostały przedstawione krzywe uczenia dla algorytmu MLP

In [ ]:
plot_learning_curve(MLPClassifier(alpha=1, max_iter=1000), hog_train, y, cv=cv, n_jobs=4)

# Wykorzystanie k najbliższych sąsiadów(KNN) w zadaniu klasyfikacji

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=6)

neigh.fit(x_train, y_train)

labels_predicted_neigh = neigh.predict(x_test)
print('Knn percentage correct: ', 100*accuracy_score(labels_predicted_neigh, y_test))
print(classification_report(y_test, labels_predicted_neigh))
print(confusion_matrix(y_test, labels_predicted_neigh))


## Poniżej zostały przedstawione krzywe uczenia dla algorytmu KNN

In [ ]:
plot_learning_curve(neigh, hog_train, y, cv=cv, n_jobs=4)

## Techniki łączeni algorytmów uczenia maszynowego 

### Głosowanie ze względu na większość głosów 